In [1]:
import numpy as np
from matplotlib import pyplot as plt

import numpy as np
import cv2
import glob
import os
import random


In [2]:
"""
Implement the number of vertical and horizontal corners
nb_vertical = ...
nb_horizontal = ...
"""
nb_vertical = 6
nb_horizontal = 9
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((nb_horizontal*nb_vertical,3), np.float32)
objp[:,:2] = np.mgrid[0:nb_vertical,0:nb_horizontal].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('rs/*.png')
assert images

# Choose only 20 random images to process
images = random.sample(images, min(20, len(images)))

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    """
    Implement findChessboardCorners here
    ret, corners = ...
    """
    ret, corners = cv2.findChessboardCorners(gray, (nb_vertical,nb_horizontal), None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nb_vertical,nb_horizontal), corners,ret)
        cv2.imshow('img',img)
        cv2.waitKey(250)

cv2.destroyAllWindows()

In [3]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
img = cv2.imread('rs/left-0001.png')
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [4]:
# the stereo images are named left-00XX.png and right-00XX.png
# select 20 random pairs for calibration
left_images = glob.glob('rs/left-*.png')
right_images = glob.glob('rs/right-*.png')
assert left_images and right_images

# select 20 random pairs for calibration
left_images = random.sample(left_images, min(20, len(left_images)))
right_images = random.sample(right_images, min(20, len(right_images)))

#undistort and crop and store into arrays
left_undistorted = []
right_undistorted = []
for fname in left_images:
    img = cv2.imread(fname)
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    left_undistorted.append(dst)
    #cv2.imwrite('calibresult.png',dst)
    cv2.imshow('calibresult',dst)
    cv2.waitKey(250)
for fname in right_images:
    img = cv2.imread(fname)
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    right_undistorted.append(dst)
    #cv2.imwrite('calibresult.png',dst)
    cv2.imshow('calibresult',dst)
    cv2.waitKey(250)
#cv2.destroyAllWindows()